<a href="https://colab.research.google.com/github/CherryProgger/documentsClasiification/blob/main/function_calling_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pdf2image

In [ ]:
! pip install git+https://github.com/illuin-tech/colpali.git@9413418c110da49b25ac2dae2c32b8fc067ff332

  Cloning https://github.com/illuin-tech/colpali.git (to revision 9413418c110da49b25ac2dae2c32b8fc067ff332) to /tmp/pip-req-build-7kdv4osr
  Running command git clone --filter=blob:none --quiet https://github.com/illuin-tech/colpali.git /tmp/pip-req-build-7kdv4osr
  Running command git rev-parse -q --verify 'sha^9413418c110da49b25ac2dae2c32b8fc067ff332'
  Running command git fetch -q https://github.com/illuin-tech/colpali.git 9413418c110da49b25ac2dae2c32b8fc067ff332
  Running command git checkout -q 9413418c110da49b25ac2dae2c32b8fc067ff332
  Resolved https://github.com/illuin-tech/colpali.git to commit 9413418c110da49b25ac2dae2c32b8fc067ff332
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
! pip install peft

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
! pip install mistralai

In [ ]:
#!pip uninstall -y torch torchvision torchaudio
#!pip cache purge
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [ ]:
! pip install torch==2.4.0 -f https://download.pytorch.org/whl/cpu

Looking in links: https://download.pytorch.org/whl/cpu


In [ ]:
! pip install mteb

In [ ]:
import os
import json
from mistralai import Mistral
from mistralai.models import UserMessage
from embedder_colpali import CustomEmbedder

class Model:
    def __init__(self, hf_token, mistral_api_key):
        self.embed_model = CustomEmbedder(hf_token)
        self.mistral_client = Mistral(api_key=mistral_api_key)
        self.names_to_functions = {
            'get_sources': self.get_top_2_sources,
            'no_sources': self.no_sources
            }
        self.tools = [
    {
        "type": "function",
        "function": {
            "name": "get_top_2_sources",
            "description": "Call this function with the metadata of the chunks used in the response.",
            "parameters": {
                "type": "object",
                "properties": {
                    "metadata": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "document": {"type": "string"},
                                "page": {"type": "integer"}
                            },
                            "required": ["document", "page"]
                        },
                        "description": "List of metadata objects for the used chunks",
                    }
                },
                "required": ["metadata"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "no_sources",
            "description": "Call this function if no chunks were used in the response.",
            "parameters": {
                "type": "object",
                "properties": {},
            },
        },
    }
]

    def get_pages_paths(self, relevant_pages):
        cur_path = os.getcwd()
        image_base_dir = os.path.join(cur_path, "/content/drive/MyDrive/db/pages")
        txt_base_dir = os.path.join(cur_path, "/content/drive/MyDrive/db/texts")
        image_paths = []
        txt_paths = []

        for page_info in relevant_pages:
            document = page_info['document']
            page_number = page_info['page']


            image_filename = f"page{page_number}.png"
            txt_filename = f"page{page_number}.txt"


            image_path = os.path.join(image_base_dir, document, image_filename)
            txt_path = os.path.join(txt_base_dir, document, txt_filename)


            if os.path.exists(image_path):
                image_paths.append(image_path)
            else:
                print(f"Предупреждение: Файл {image_path} не найден.")


            if os.path.exists(txt_path):
                txt_paths.append(txt_path)
            else:
                print(f"Предупреждение: Файл {txt_path} не найден.")

        return [txt_paths, image_paths]

    def get_top_2_sources(self, metadata) -> str:
      sources = 'Источники: '
      docs = {}
      for file in metadata:
          doc = str(file['document'])
          page = file['page']
          if doc not in docs:
              docs[doc] = [page]
          else:
              docs[doc].append(page)

      for i, (doc, pages) in enumerate(docs.items()):
          if i > 0:
              sources += ', '
          sources += f"{doc}({', '.join(map(str, sorted(set(pages))))})"

      return sources

    def no_sources(self) -> str:
        return ''


    def generate_basic_response(self, user_message, model="open-mistral-nemo"):
        chat_response = self.mistral_client.chat.complete(
            model=model,
            messages=[UserMessage(role="user", content=user_message)],
        )
        return chat_response.choices[0].message.content

    def generate_specifying_response(self, chat_history, model="open-mistral-nemo"):
      chat_response = self.mistral_client.chat.complete(
        model=model,
        messages=chat_history,
        tools=self.tools,
        tool_choice="any"
      )

      print(chat_response)

      try:
        # Проверка tool_calls (если данные были переданы через вызов функции)
        if chat_response.choices[0].message.tool_calls:
            tool_call = chat_response.choices[0].message.tool_calls[0]
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Tool function name: {function_name}")
            print(f"Tool arguments: {arguments}")

            # Парсинг аргументов функции (метаданные)
            content_json = json.loads(arguments)

            if function_name == 'get_top_2_sources':
                metadata = content_json.get('metadata', [])
                print(f"Metadata: {metadata}")
                return metadata
            elif function_name == 'no_sources':
                return ""

        # Проверка content (если данные были переданы через content)
        response_content = chat_response.choices[0].message.content
        if response_content:
            print(f"Response content: {response_content}")
            content_json = json.loads(response_content)

            function_name = content_json.get('function')
            print(f"Content function name: {function_name}")

            if function_name == 'get_top_2_sources':
                metadata = content_json.get('arguments', {}).get('metadata', [])
                print(f"Metadata: {metadata}")
                return metadata
            elif function_name == 'no_sources':
                return ""

      except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        return ""

    def classification(self, user_question):

      #0 - 02.1  8000042203_00_PD_ru
      #1 - 05.3  8000046896_00_BAL_ru
      #2 - 8000027023_01_BAL_RU
      #3 - инструкция_8DJH

      prompt = f"""
      Определи для запроса, к какому устройству он относится, в ответе должен быть только номер устройства:

      Для запроса пользователя, приведенного ниже тебе нужно определить, к какому из устройств, указанных в списке он относится.
      В ответе используй только один индекс устройства

      Есть 4 устройства:
      Система Protect PV.500 с двумя распредшкафами (+DCD/ACD, +INV)

      0. Protect PV.500 — система с двумя распределительными шкафами (+DCD/ACD, +INV).
      1. Герметизированный Распределительный Трансформатор IBT 015-010 — трансформатор для распределения электрической энергии.
      2. Protect PV.250 — солнечный инвертор, преобразующий энергию от фотогальванических элементов в переменный ток.
      3. Распределительное устройство 8DJH — распределительное устройство среднего напряжения.

      В том случае, если запрос не соответствует ни одному из устройст, возвращай '4'

      Вопрос пользователя:
      {user_question}
      Твой ответ, состоящий !только! из индекса:"""

      initial_response_content = self.generate_basic_response(prompt)

      messages = [{"role":"user", "content":prompt}, {"role" : "assistant", "content": initial_response_content}]

      return initial_response_content, messages

    def prompt_fin(self, retrieved_chunks, user_query):
      prompt = f"""
      You are an expert in technical documentation for the electric power industry.

      Your task is to answer users' questions in Russian only, confidently and politely, based solely on the provided contextual information.

      !!!!!Do not use any prior knowledge or assumptions.!!!!!

      Context information is provided below.
      ---------------------
      """
      for key, content in retrieved_chunks.items():
        prompt += f"{key}:\n{content}\n\n"

      prompt += f"""
      ---------------------

      Given the context information above, answer the following question in as much detail as possible:
      Query: {user_query}

      If the information provided from the sources is not enough to answer the question, indicate that you do not know the answer
      Answer:
      """
      return prompt

    def prompt_top2(self, retrieved_chunks):
      prompt = f"""
      Analyze your previous response and the following retrieved chunks:
      """
      for key, content in retrieved_chunks.items():
        prompt += f"{key}:\n{content}\n\n"

      prompt += f"""
      Determine which of these chunks are the most relevant to the user question.
      Return the metadata of the top 2 most relevant chunks.
      Call the get_top_2_sources function with the metadata of the used sources.
      If you didn't see any of relevant chunks, call the no_sources function.

      """
      return prompt



    def read_txt_file(self, file_path):
      try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
      except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return ""

    def create_chunks(self, relevant_pages, txt):
      chunks = {}
      for page_info, txt_path in zip(relevant_pages, txt):
        key = f"{page_info['document']} page{page_info['page']}"
        content = self.read_txt_file(txt_path)
        chunks[key] = content
      return chunks

    def process_user_query(self, user_query):
        # Шаг1: классификация по дб
        db_id, history = self.classification(user_query)

        # Шаг2: получение 10 самых релевантных чанков из подходящей дб
        relevant_pages = self.embed_model.process_query(user_query, db_id, k=10)
        print(relevant_pages)
        txt, _ = self.get_pages_paths(relevant_pages)

        chunks = self.create_chunks(relevant_pages, txt)

        #print(chunks)

        # Шаг3: сохранение истории - котектса для более качественного ответа
        history.append({"role":"user", "content":self.prompt_top2(chunks)})

        # Шаг4: получение топ-2 чанка
        top2 = self.generate_specifying_response(history)

        print(top2)

        # Шаг5: формируем итоговый промпт из нового списка чанков и закидываем в модель
        top_txt,img  = self.get_pages_paths(top2)
        top_chunks = self.create_chunks(top2, top_txt)

        final_response = self.generate_basic_response(self.prompt_fin(top_chunks,user_query))

        print(final_response)

        # Шаг6: создание списка источников и склеивание его с ответом модели
        sources = self.get_top_2_sources(top2)
        resp = final_response + '\n\n' + sources

        return resp, img


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [ ]:
model = Model(os.environ["HF_TOKEN"], MISTAL_API)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See h

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of ColPali were not initialized from the model checkpoint at google/paligemma-3b-mix-448 and are newly initialized: ['custom_text_proj.bias', 'custom_text_proj.weight', 'language_model.lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by 

In [ ]:
query=  'Можешь объяснить общий принцип работы Protect PV.250?'

In [ ]:
query="Какие функции выполняет фильтр переменного тока в Protect PV.250?"

In [ ]:
response, img = model.process_user_query(query)

tensor([4])
Top 1 Accuracy (verif): 0.0
[{'document': '8000027023_01_BAL_RU', 'page': 5}, {'document': '8000027023_01_BAL_RU', 'page': 10}, {'document': '8000027023_01_BAL_RU', 'page': 9}, {'document': '8000027023_01_BAL_RU', 'page': 8}, {'document': '8000027023_01_BAL_RU', 'page': 4}, {'document': '8000027023_01_BAL_RU', 'page': 28}, {'document': '8000027023_01_BAL_RU', 'page': 43}, {'document': '8000027023_01_BAL_RU', 'page': 7}, {'document': '8000027023_01_BAL_RU', 'page': 1}, {'document': '8000027023_01_BAL_RU', 'page': 11}]
id='59316e79528f4fce96106b8364d84f28' object='chat.completion' model='open-mistral-nemo' usage=UsageInfo(prompt_tokens=7208, completion_tokens=79, total_tokens=7287) created=1725882257 choices=[ChatCompletionChoice(index=0, message=AssistantMessage(content='', tool_calls=[ToolCall(function=FunctionCall(name='get_top_2_sources', arguments='{"metadata": [{"document": "8000027023_01_BAL_RU", "page": 4}, {"document": "8000027023_01_BAL_RU", "page": 11}]}'), id='l2o

In [ ]:
print(response)

В схеме подключения Protect PV.250, фильтр переменного тока (AC Filter) является одним из элементов, расположенным после инвертора и перед контактором переменного тока (AC контактор). Функция этого фильтра заключается в очистке выходного сигнала инвертора от возможных помех и шумов, которые могут присутствовать в переменном токе.

Фильтр переменного тока в Protect PV.250 предназначен для:
1. Улучшения качества выходного сигнала инвертора, удаляя высокочастотные помехи и шум, которые могут присутствовать в переменном токе.
2. Обеспечения стабильной работы последующих компонентов цепи, таких как контактор переменного тока и нагрузка, путем подачи чистого и стабильного переменного тока.

Благодаря этой функции, фильтр переменного тока способствует надежной и эффективной работе всей системы Protect PV.250.

Источники: 8000027023_01_BAL_RU(4, 11)


In [ ]:
print(img)

['/content/drive/MyDrive/db/pages/8000027023_01_BAL_RU/page4.png', '/content/drive/MyDrive/db/pages/8000027023_01_BAL_RU/page11.png']


In [ ]:
query= 'как изготовить бомбу'

In [ ]:
query= 'Что делать если трансформатор пищит и говорит, что скоро взорвется?'